# Search for AIMA 4th edition

Implementation of search algorithms and search problems for AIMA.

# Problems and Nodes

We start by defining the abstract class for a `Problem`; specific problem domains will subclass this. To make it easier for algorithms that use a heuristic evaluation function, `Problem` has a default `h` function (uniformly zero), and subclasses can define their own default `h` function.

We also define a `Node` in a search tree, and some functions on nodes: `expand` to generate successors; `path_actions` and `path_states`  to recover aspects of the path from the node.  

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys, os
import time

sys.path.append(os.getcwd())
#import bisect
from bisect import *

from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# Queues

First-in-first-out and Last-in-first-out queues, and a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.

In [3]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms: Best-First

Best-first search with various *f(n)* functions gives us different search algorithms. Note that A\*, weighted A\* and greedy search can be given a heuristic function, `h`, but if `h` is not supplied they use the problem's default `h` function (if the problem does not define one, it is taken as *h(n)* = 0).

In [4]:

def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    seconds = time.time()
    NODES_EXPANDED = 0
    STATES_REEXPANDED = 0
    DUMMY_NODES = 0
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    expanded = {}
    while frontier:
        # node extraction, no  dummy nodes control 
        node = frontier.pop()
        # Reexpansion control, number of states reexpanded
        """
        while not ok:
            node = frontier.pop()
            if node.path_cost > reached[node.state].path_cost:
                DUMMY_NODES+=1
            else:
                ok = True
        """
        while node.path_cost > reached[node.state].path_cost:
            DUMMY_NODES+=1
            node = frontier.pop()
        # counting expansions  
        if node.state in expanded:
            STATES_REEXPANDED +=1
        else:
            expanded[node.state] = True  
        NODES_EXPANDED+=1
        # Monitoring expanded states and f-values
        #print(node.state, "f()= ", f(node))
        #if goal, show statistics
        if problem.is_goal(node.state):
            #with open('astar-data.csv','a') as f:
            #    f.write(f'{time.time()-seconds};{NODES_EXPANDED};{len(reached)};{len(frontier)};{node.path_cost};{len(node)};{STATES_REEXPANDED};{DUMMY_NODES}\n')
            print("Time Taken: ", time.time()-seconds)
            print("Nodes Expanded: ", NODES_EXPANDED)
            print("States Reached: ", len(reached))
            print("Nodes in Frontier: ", len(frontier))
            print("Solution Cost: ", node.path_cost)
            print("Solution Length: ", len(node))
            print("Reexpanded States: ", STATES_REEXPANDED)
            print("Dummy Nodes: ", DUMMY_NODES)
            return node
        # node expansion

        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))

#PEA*
#Al menos considerar 2 instancias
def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



# Other Search Algorithms

Here are the other search algorithms:

In [5]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result != failure:
                return result
        return failure

In [6]:
#path_states(depth_first_recursive_search(r2))

# Bidirectional Best-First Search

In [7]:
def bidirectional_best_first_search(problem_f, f_f, problem_b, f_b, terminated):
    node_f = Node(problem_f.initial)
    node_b = Node(problem_f.goal)
    frontier_f, reached_f = PriorityQueue([node_f], key=f_f), {node_f.state: node_f}
    frontier_b, reached_b = PriorityQueue([node_b], key=f_b), {node_b.state: node_b}
    solution = failure
    while frontier_f and frontier_b and not terminated(solution, frontier_f, frontier_b):
        def S1(node, f):
            return str(int(f(node))) + ' ' + str(path_states(node))
        print('Bi:', S1(frontier_f.top(), f_f), S1(frontier_b.top(), f_b))
        if f_f(frontier_f.top()) < f_b(frontier_b.top()):
            solution = proceed('f', problem_f, frontier_f, reached_f, reached_b, solution)
        else:
            solution = proceed('b', problem_b, frontier_b, reached_b, reached_f, solution)
    return solution

def inverse_problem(problem):
    if isinstance(problem, CountCalls):
        return CountCalls(inverse_problem(problem._object))
    else:
        inv = copy.copy(problem)
        inv.initial, inv.goal = inv.goal, inv.initial
        return inv

In [8]:
def bidirectional_uniform_cost_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        n_f, n_b = frontier_f.top(), frontier_b.top()
        return g(n_f) + g(n_b) > g(solution)
    return bidirectional_best_first_search(problem_f, g, inverse_problem(problem_f), g, terminated)

def bidirectional_astar_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        nf, nb = frontier_f.top(), frontier_b.top()
        return g(nf) + g(nb) > g(solution)
    problem_f = inverse_problem(problem_f)
    return bidirectional_best_first_search(problem_f, lambda n: g(n) + problem_f.h(n),
                                           problem_b, lambda n: g(n) + problem_b.h(n), 
                                           terminated)
   

def proceed(direction, problem, frontier, reached, reached2, solution):
    node = frontier.pop()
    for child in expand(problem, node):
        s = child.state
        print('proceed', direction, S(child))
        if s not in reached or child.path_cost < reached[s].path_cost:
            frontier.add(child)
            reached[s] = child
            if s in reached2: # Frontiers collide; solution found
                solution2 = (join_nodes(child, reached2[s]) if direction == 'f' else
                             join_nodes(reached2[s], child))
                #print('solution', path_states(solution2), solution2.path_cost, 
                # path_states(child), path_states(reached2[s]))
                if solution2.path_cost < solution.path_cost:
                    solution = solution2
    return solution

S = path_states

#A-S-R + B-P-R => A-S-R-P + B-P
def join_nodes(nf, nb):
    """Join the reverse of the backward node nb to the forward node nf."""
    #print('join', S(nf), S(nb))
    join = nf
    while nb.parent is not None:
        cost = join.path_cost + nb.path_cost - nb.parent.path_cost
        join = Node(nb.parent.state, join, nb.action, cost)
        nb = nb.parent
        #print('  now join', S(join), 'with nb', S(nb), 'parent', S(nb.parent))
    return join
    
   

In [9]:
#A , B = uniform_cost_search(r1), uniform_cost_search(r2)
#path_states(A), path_states(B)

In [10]:
#path_states(append_nodes(A, B))

# TODO: RBFS

# Problem Domains

Now we turn our attention to defining some problem domains as subclasses of `Problem`.

# Symmetric TSP

![](romania.png)

En el TSP simetrico, los estados son pares de la forma [CiudadesVisitadas, CiudadActual]. El estado inicial es [ConjuntoVacio, [A]], el estado objetivo es de la forma [TodasLasCiudades, [A]]. Los sucesores de un estado se generan a partir de las ciudades no visitadas, uno por cada una. El coste de una regla es el coste entre la ciudad actual y la nueva ciudad visitada que pasa a ser la actual

In [11]:
class EstadoTSP:
    def __init__(self, actual, visitadas):
        self.actual = actual
        self.visitadas = visitadas
        lista = visitadas.copy()
        lista.append(actual)
        #Necesitamos que sea hasheable por lo que hacemos que sea inmutable haciendo que sea una tupla
        self.state = tuple(lista)
       # self.hash = hash(self.state)
    def __hash__(self):
        return hash(self.state)
        #return self.hash
    def __eq__(self,other):
        return self.actual == other.actual and self.visitadas == other.visitadas
    def __str__(self):
        return " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]"  


In [12]:
import copy
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        # print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

    def residual(self, nodes,node):
        grafocopia = copy.deepcopy(self)
        
        # Filtra las distancias para mantener solo las ciudades en 'nodes'
        new_dist = []
        for index_i,i in enumerate(nodes):
            row = []
            for j in nodes:
                #(A{B}C) El grafo residual no contiene arista entre A y C, se comprueba de esta forma
                if len(nodes)>2 and ((i == 0 and j == node.state.actual) or (i == node.state.actual and j==0)):
                    row.append(float('inf'))
                else:
                    row.append(grafocopia.Dist[i][j])
            new_dist.append(row)

        grafocopia.Dist=new_dist
        # Actualiza la lista de ciudades
        grafocopia.ciudades = nodes
        
        return grafocopia
    
    def get_arcos_residual(self, nodes,node):
        arcos =[]
        for index_i,i in enumerate(nodes):
            for j in nodes[index_i+1:]:
                if i!=j and not ((i == 0 and j == node.state.actual)):
                    arcos.append(self.Dist[i][j])
                elif len(nodes)<=2:
                    arcos.append(self.Dist[i][j])
        return arcos
    
    def get_arcos_con_nodos_residual(self, nodes,node):
        arcos =[]
        for index_i,i in enumerate(nodes):
            for j in nodes[index_i+1:]:
                if i!=j and not ((i == 0 and j == node.state.actual)):
                    arcos.append((self.Dist[i][j],i,j))
                elif len(nodes)<=2:
                    arcos.append((self.Dist[i][j],i,j))
        return arcos

        


In [38]:
class SymmetricTSP(Problem):
    """El problema consiste en encontrar un camino hamiltoniano en un grafo no dirigido 
    y totalmente conectado, con costes positivos en los ejes. Las ciudades son 0,...,N-1
    La ciudad de partida es la 0"""

    def __init__(self, grafo):
        self.grafo = grafo
        self.initial = EstadoTSP(0,[])
        #print(self.__dir__initial)

    def actions(self, state):
        """Una accion por cada ciudad no visitada distinta de la actual, si todas visitadas -1"""
        noVisitadas = []
        for c in self.grafo.ciudades: 
            if c not in state.visitadas and c != state.actual:
                noVisitadas.append(c) 
        if len(noVisitadas)==0:
            noVisitadas = [-1]
        return noVisitadas
    
    def result(self, state, action): 
        setVisitadas = list(state.visitadas)
        insort(setVisitadas,state.actual)
        return EstadoTSP(action,setVisitadas)
    
    def is_goal(self, state):        return state.actual == -1

    def action_cost(self, s, a, s1): 
        if (len(s.visitadas)==self.grafo.N-1):
            return self.grafo.Dist[s.actual][0]
        else:
            return self.grafo.Dist[s.actual][a]
    
    def calcular_ciudades_residuales(self,node):
        residuales = []
        k=0
        #Calculamos las ciudades no visitadas incluyendo la principal y la actual
        for c in self.grafo.ciudades:
            if c not in node.state.visitadas or c ==0 or c == node.state.actual:
                residuales.append(c)
            if c in node.state.visitadas or c == node.state.actual:
                k+=1
        return residuales,k
        
    def h(self,node):
        return self.h_mst(node)

    def h0(self,node):
        return self.grafo.N - len(node.state.visitadas) 
    
    def h1(self,node):
        porabandonar = []
        cotainferior = 0
        for c in self.grafo.ciudades: 
            if c not in node.state.visitadas or c == node.state.actual:
                porabandonar.append(c)
        for ciudad in porabandonar:
            aux = []
            for ciudad2 in porabandonar:
                if ciudad!=ciudad2:
                    aux.append(self.grafo.Dist[ciudad][ciudad2])
            #En caso de que solo quede por abandonar la ultima ciudad contemplamos el camino entre esa ciudad y el inicial
            if len(porabandonar) > 1:
                cotainferior+= min(aux)
            else:
                cotainferior+= self.grafo.Dist[ciudad][0]
        return cotainferior
    
    def h2(self,node):
        residuales,visitadas = self.calcular_ciudades_residuales(node)
        residual=self.grafo.get_arcos_residual(residuales,node)
        residual.sort()
        return sum(residual[:int(self.grafo.N)-visitadas+1])
    
    def h3(self,node):
        residuales,visitadas = self.calcular_ciudades_residuales(node)
        arcos=self.grafo.get_arcos_con_nodos_residual(residuales,node)
        arcos = sorted(arcos, key=lambda x: x[0]) #Ordenamos por coste
        sumaArcos=[]
        ciudadesporannadir = residuales.copy()
        arcos_ciudades_annadidas = []
        for conexiones in arcos:
            if conexiones[1] in ciudadesporannadir or conexiones[2] in ciudadesporannadir:
                sumaArcos.append(conexiones)
                if conexiones[1] in ciudadesporannadir:
                    ciudadesporannadir.remove(conexiones[1])
                if conexiones[2] in ciudadesporannadir:
                    ciudadesporannadir.remove(conexiones[2])
                if len(ciudadesporannadir) ==0:
                    arcos_por_annadir = int(self.grafo.N)-visitadas+1 - len(sumaArcos)
                    if len(arcos_ciudades_annadidas) >= arcos_por_annadir:
                        sumaArcos.extend(arcos_ciudades_annadidas[:arcos_por_annadir])
                        break
            else:
                arcos_ciudades_annadidas.append(conexiones)
                if len(ciudadesporannadir) ==0:
                    arcos_por_annadir = int(self.grafo.N)-visitadas+1 - len(sumaArcos)
                    #Si se puede annadir el numero de ciudades restantes por annadir se hace y se sale del bucle
                    if len(arcos_ciudades_annadidas) >= arcos_por_annadir:
                        sumaArcos.extend(arcos_ciudades_annadidas[:arcos_por_annadir])
                        break
        return sum(i[0] for i in sumaArcos)


    #Para comparar heuristicos nos fijamos en los nodos expandidos y en el tiempo

    def h_mst(self, node):
        novisitadas,_ = self.calcular_ciudades_residuales(node)
        residual = self.grafo.residual(novisitadas,node)
        # Inicializar el árbol con un vértice arbitrario
        arbol_expansion = []
        arbol_expansion.append((0, 0))
        
        while len(arbol_expansion) < len(residual.ciudades):
            # Encontrar el lado de menor distancia desde el árbol actual
            min_peso = float('inf')
            min_lado = None

            for arcos in arbol_expansion:
                for ciudad in range(len(residual.ciudades)):
                    if ciudad not in [x[1] for x in arbol_expansion]:
                        peso = residual.Dist[arcos[1]][ciudad]
                        if peso < min_peso:
                            min_peso = peso
                            min_lado = (arcos[1], ciudad)

            if min_lado:
                arbol_expansion.append(min_lado)

        costo_arbol_expansion = sum(residual.Dist[ciudad1][ciudad2] for ciudad1, ciudad2 in arbol_expansion)
        return costo_arbol_expansion


         

                    
        
        


In [39]:
# Datos de instancias en formato EDGE_WEIGHT_TYPE de la TSPLIB

prueba = [ 
    0,
    1, 0,
    6, 2, 0,
    4, 5, 3, 0,
]

gr21_commented_4 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    #450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    #170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    #240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    #380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

fri26_commented_5 = [
    0,
    83, 0,
    93, 40, 0,
    129, 53, 42, 0,
    133, 62, 42, 11, 0,
    139, 64, 49, 11, 9, 0,
    151, 91, 59, 46, 35, 39, 0,
    169, 116, 81, 72, 61, 65, 26, 0,
    135, 93, 54, 65, 55, 63, 34, 37, 0,
    114, 84, 44, 70, 62, 71, 52, 59, 22, 0,
    110, 95, 58, 88, 82, 90, 71, 75, 39, 20, 0,
    98, 98, 64, 100, 95, 103, 88, 92, 56, 36, 18, 0,
    99, 89, 54, 89, 84, 92, 77, 83, 47, 26, 11, 11, 0,
    95, 68, 31, 66, 62, 71, 63, 76, 40, 20, 27, 34, 23, 0,
    81, 67, 36, 76, 74, 82, 78, 91, 55, 34, 32, 31, 24, 15, 0,
    152, 127, 86, 102, 93, 100, 66, 54, 37, 43, 42, 56, 53, 62, 73, 0,
    159, 156, 117, 142, 133, 141, 110, 98, 78, 74, 61, 63, 68, 87, 92, 44, 0,
    181, 175, 135, 156, 146, 153, 119, 103, 91, 91, 80, 85, 89, 106, 112, 54, 22, 0,
    172, 152, 112, 127, 117, 124, 88, 70, 62, 68, 64, 75, 74, 87, 96, 26, 34, 33, 0,
    185, 165, 125, 139, 128, 135, 98, 78, 74, 82, 77, 87, 87, 100, 109, 39, 38, 29, 13, 0,
    147, 160, 124, 155, 148, 156, 130, 122, 96, 86, 68, 62, 71, 93, 93, 68, 30, 46, 63, 68, 0,
    #157, 180, 147, 180, 173, 181, 156, 148, 122, 111, 92, 83, 93, 116, 113, 94, 53, 64, 87, 90, 26, 0,
    #185, 223, 193, 228, 222, 230, 206, 198, 172, 160, 140, 129, 140, 163, 158, 144, 102, 107, 135, 136, 77, 50, 0,
    #220, 268, 241, 278, 272, 280, 257, 250, 223, 210, 190, 178, 189, 212, 205, 196, 154, 157, 186, 186, 128, 102, 51, 0,
    #127, 179, 157, 197, 194, 202, 188, 188, 155, 136, 116, 100, 111, 132, 122, 139, 109, 125, 141, 148, 80, 65, 64, 93, 0,
    #181, 197, 161, 190, 182, 190, 160, 148, 128, 121, 103, 99, 107, 130, 130, 95, 51, 51, 81, 79, 37, 27, 58, 107, 90, 0
]

instance_gr17 = SymmetricTSP(Grafo(gr17))
instance_gr21_commented_4 = SymmetricTSP(Grafo(gr21_commented_4))
instance_gr21 = SymmetricTSP(Grafo(gr21))
instance_fri26_commented_5 = SymmetricTSP(Grafo(fri26_commented_5))

# with open('astar-h2-convergence.csv','a') as f:
#     f.write(f'gr17\n')
# path_actions(astar_search(instance_gr17,h=instance_gr17.h2))
# with open('astar-h2-convergence.csv','a') as f:
#     f.write(f'gr21-4\n')
# path_actions(astar_search(instance_gr21_commented_4, h=instance_gr21_commented_4.h2))
# with open('astar-h2-convergence.csv','a') as f:
#     f.write(f'gr21\n')
# path_actions(astar_search(instance_gr21, h=instance_gr21.h2))
# with open('astar-h2-convergence.csv','a') as f:
#     f.write(f'fri26-5\n')
# path_actions(astar_search(instance_fri26_commented_5, h=instance_fri26_commented_5.h2))

#for weight in [1.2,1.4,1.6,1.8,1.99]:
# with open('astar-data.csv','a') as f:
#     f.write(f'h1\n')
# for name, ins in {"gr17":instance_gr17,"gr21_commented_4":instance_gr21_commented_4,"gr21":instance_gr21,"fri26_commented_5":instance_fri26_commented_5}.items():
#     for _ in range(10):  
#         path_actions(astar_search(ins, h=ins.h1))
# with open('astar-data.csv','a') as f:
#     f.write(f'h2\n')
# for name, ins in {"gr17":instance_gr17,"gr21_commented_4":instance_gr21_commented_4,"gr21":instance_gr21,"fri26_commented_5":instance_fri26_commented_5}.items():
#     for _ in range(10):  
#         path_actions(astar_search(ins, h=ins.h2))
# with open('astar-data.csv','a') as f:
#     f.write(f'h3\n')
# for name, ins in {"gr17":instance_gr17,"gr21_commented_4":instance_gr21_commented_4,"gr21":instance_gr21,"fri26_commented_5":instance_fri26_commented_5}.items():
#     for _ in range(10):  
#         path_actions(astar_search(ins, h=ins.h3))
# with open('astar-data.csv','a') as f:
#     f.write(f'h_mst\n')
    #"gr17":instance_gr17,"gr21_commented_4":instance_gr21_commented_4,"gr21":instance_gr21,
# for name, ins in {"fri26_commented_5":instance_fri26_commented_5}.items():
#     for _ in range(10):  
#         path_actions(astar_search(ins, h=ins.h_mst))


Time Taken:  55.810632944107056
Nodes Expanded:  30413
States Reached:  113140
Nodes in Frontier:  106981
Solution Cost:  2085
Solution Length:  17
Reexpanded States:  0
Dummy Nodes:  5927
Time Taken:  1.600001573562622
Nodes Expanded:  269
States Reached:  1873
Nodes in Frontier:  1941
Solution Cost:  2369
Solution Length:  17
Reexpanded States:  0
Dummy Nodes:  49
Time Taken:  66.74669766426086
Nodes Expanded:  7202
States Reached:  60184
Nodes in Frontier:  61990
Solution Cost:  2707
Solution Length:  21
Reexpanded States:  0
Dummy Nodes:  596
Time Taken:  9.617000579833984
Nodes Expanded:  788
States Reached:  7288
Nodes in Frontier:  7488
Solution Cost:  660
Solution Length:  21
Reexpanded States:  0
Dummy Nodes:  40


[14, 13, 9, 10, 12, 11, 20, 16, 17, 19, 18, 15, 8, 7, 6, 4, 5, 3, 2, 1, -1]

# Route Finding Problems

![](romania.png)

In a `RouteProblem`, the states are names of "cities" (or other locations), like `'A'` for Arad. The actions are also city names; `'Z'` is the action to move to city `'Z'`. The layout of cities is given by a separate data structure, a `Map`, which is a graph where there are vertexes (cities), links between vertexes, distances (costs) of those links (if not specified, the default is 1 for every link), and optionally the 2D (x, y) location of each city can be specified. A `RouteProblem` takes this `Map` as input and allows actions to move between linked cities. The default heuristic is straight-line distance to the goal, or is uniformly zero if locations were not given.

In [15]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [16]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [17]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
r1 = RouteProblem('A', 'B', map=romania)
r2 = RouteProblem('N', 'L', map=romania)
r3 = RouteProblem('E', 'T', map=romania)
r4 = RouteProblem('O', 'M', map=romania)

In [18]:
path_states(uniform_cost_search(r1)) # Lowest-cost path from Arab to Bucharest

Time Taken:  0.0
Nodes Expanded:  13
States Reached:  13
Nodes in Frontier:  1
Solution Cost:  418
Solution Length:  4
Reexpanded States:  0
Dummy Nodes:  0


['A', 'S', 'R', 'P', 'B']

In [19]:
path_states(breadth_first_search(r1)) # Breadth-first: fewer steps, higher path cost

['A', 'S', 'F', 'B']

# Grid Problems

A `GridProblem` involves navigating on a 2D grid, with some cells being impassible obstacles. By default you can move to any of the eight neighboring cells that are not obstacles (but in a problem instance you can supply a `directions=` keyword to change that). Again, the default heuristic is straight-line distance to the goal. States are `(x, y)` cell locations, such as `(4, 2)`, and actions are `(dx, dy)` cell movements, such as `(0, -1)`, which means leave the `x` coordinate alone, and decrement the `y` coordinate by 1.

In [20]:
class GridProblem(Problem):
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(15, 30), goal=(130, 30), obstacles=(), **kwds):
        Problem.__init__(self, initial=initial, goal=goal, 
                         obstacles=set(obstacles) - {initial, goal}, **kwds)

    directions = [(-1, -1), (0, -1), (1, -1),
                  (-1, 0),           (1,  0),
                  (-1, +1), (0, +1), (1, +1)]
    
    def action_cost(self, s, action, s1): return straight_line_distance(s, s1)
    
    def h(self, node): return straight_line_distance(node.state, self.goal)
                  
    def result(self, state, action): 
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state
    
    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state
        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles
    
class ErraticVacuum(Problem):
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = dict(suck=    {1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
                 forward= {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
                 backward={1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}})

In [21]:
# Some grid routing problems

# The following can be used to create obstacles:
    
def random_lines(X=range(15, 130), Y=range(60), N=150, lengths=range(6, 12)):
    """The set of cells in N random lines of the given lengths."""
    result = set()
    for _ in range(N):
        x, y = random.choice(X), random.choice(Y)
        dx, dy = random.choice(((0, 1), (1, 0)))
        result |= line(x, y, dx, dy, random.choice(lengths))
    return result

def line(x, y, dx, dy, length):
    """A line of `length` cells starting at (x, y) and going in (dx, dy) direction."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

random.seed(42) # To make this reproducible

frame = line(-10, 20, 0, 1, 20) | line(150, 20, 0, 1, 20)
cup = line(102, 44, -1, 0, 15) | line(102, 20, -1, 0, 20) | line(102, 44, 0, -1, 24)

d1 = GridProblem(obstacles=random_lines(N=100) | frame)
d2 = GridProblem(obstacles=random_lines(N=150) | frame)
d3 = GridProblem(obstacles=random_lines(N=200) | frame)
d4 = GridProblem(obstacles=random_lines(N=250) | frame)
d5 = GridProblem(obstacles=random_lines(N=300) | frame)
d6 = GridProblem(obstacles=cup | frame)
d7 = GridProblem(obstacles=cup | frame | line(50, 35, 0, -1, 10) | line(60, 37, 0, -1, 17) | line(70, 31, 0, -1, 19))

# 8 Puzzle Problems

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.




In [22]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """
    def __init__(self, initial, goal=(1, 2, 3, 8, 0, 4, 7, 6, 5)):
    #def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can be moved to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    # action cost other than default 1
    #def action_cost(self, s, a, s1): return math.exp(s[a]);
    #def action_cost(self, s, a, s1): return s[a]

    
    # heuristic functions
    def h0(self, node):
        """The null heuristic."""
        return 0

    def h1(self, node):
        """The misplaced tiles heuristic. It has a bug as it counts 0 as avalid tilde!!"""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic for goal (0, 1, 2, 3, 4, 5, 6, 7, 8)"""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
                   # g = 0, 1, ..., 8 (tuple positions, or tildes in goal.state)
                   # s = gth tilde in the node.state
                   # X[g],Y[g] = board position of tilde s in node.state
                   # X[s],Y[s] = board position of tilde s in goal.state = (0, 1, 2, 3, 4, 5, 6, 7, 8)

    def h2_2(self,node):
        sum=0
        for i in range(9):
            if(i!=0):
                #Obtenemos la posicion actual de la i(numero)
                X1,Y1 = (node.state.index(i)%3,node.state.index(i)//3)
                #Obtenemos la posicion objetivo de la i
                Xg,Yg = (self.goal.index(i)%3,self.goal.index(i)//3)
                #Calculamos la distancia y la sumamos
                sum+= abs(X1-Xg) + abs(Y1-Yg)
        return sum
    
    def h3(self, node):
        """Heuristico que solo tiene en cuenta las casillas que están a distancia 2 del objetivo"""
        sum = 0
        for i in range(9):
            if(i!=0):
                #Obtenemos la posicion actual de la i(numero)
                X1,Y1 = (node.state.index(i)%3,node.state.index(i)//3)
                #Obtenemos la posicion objetivo de la i
                Xg,Yg = (self.goal.index(i)%3,self.goal.index(i)//3)
                distancia = abs(Xg - X1) + abs(Yg - Y1)
                if distancia != 2:
                    sum += distancia
        return sum
    
    def h4(self, node):
        """Heuristico que comprueba si la distancia es par"""
        sum = 0
        for i in range(9):
            if(i!=0):
                #Obtenemos la posicion actual de la i(numero)
                X1,Y1 = (node.state.index(i)%3,node.state.index(i)//3)
                #Obtenemos la posicion objetivo de la i
                Xg,Yg = (self.goal.index(i)%3,self.goal.index(i)//3)
                distancia = abs(Xg - X1) + abs(Yg - Y1)
                if distancia % 2 == 0:
                    sum += distancia * 0.5
                else:
                    sum+= distancia
        return sum

   
    
    # Actual heuristic
    def h(self, node): 
       return self.h4(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b and a != 0 for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')



In [23]:
# Some specific EightPuzzle problems, reachable from (1, 2, 3, 8, 0, 4, 7, 6, 5)

e1 = EightPuzzle((5, 6, 7, 2, 8, 4, 0, 3, 1))
e2 = EightPuzzle((5, 6, 7, 4, 0, 8, 3, 2, 1))
e3 = EightPuzzle((5, 4, 7, 6, 0, 3, 8, 2, 1))
e4 = EightPuzzle((3, 8, 7, 4, 0, 6, 5, 2, 1))
e5 = EightPuzzle((5, 6, 3, 4, 0, 2, 7, 8, 1))
#e6 = EightPuzzle((8, 2, 0, 3, 4, 5, 6, 1, 7))
#e7 = EightPuzzle((8, 0, 3, 2, 4, 5, 6, 1, 7))
#e8 = EightPuzzle((6, 3, 2, 5, 8, 1, 4, 0, 7))

In [24]:
# Solve an 8 puzzle problem and print out each state 
#for s in path_states(astar_search(e5)):
for s in path_states(breadth_first_bfs(e5)):
    print(board8(s))

Time Taken:  2.302377700805664
Nodes Expanded:  181320
States Reached:  181440
Nodes in Frontier:  120
Solution Cost:  30
Solution Length:  30
Reexpanded States:  0
Dummy Nodes:  0
5 6 3
4 _ 2
7 8 1

5 _ 3
4 6 2
7 8 1

5 3 _
4 6 2
7 8 1

5 3 2
4 6 _
7 8 1

5 3 2
4 _ 6
7 8 1

5 3 2
_ 4 6
7 8 1

_ 3 2
5 4 6
7 8 1

3 _ 2
5 4 6
7 8 1

3 4 2
5 _ 6
7 8 1

3 4 2
_ 5 6
7 8 1

3 4 2
7 5 6
_ 8 1

3 4 2
7 5 6
8 _ 1

3 4 2
7 5 6
8 1 _

3 4 2
7 5 _
8 1 6

3 4 2
7 _ 5
8 1 6

3 4 2
7 1 5
8 _ 6

3 4 2
7 1 5
_ 8 6

3 4 2
_ 1 5
7 8 6

3 4 2
1 _ 5
7 8 6

3 4 2
1 8 5
7 _ 6

3 4 2
1 8 5
7 6 _

3 4 2
1 8 _
7 6 5

3 4 _
1 8 2
7 6 5

3 _ 4
1 8 2
7 6 5

_ 3 4
1 8 2
7 6 5

1 3 4
_ 8 2
7 6 5

1 3 4
8 _ 2
7 6 5

1 3 4
8 2 _
7 6 5

1 3 _
8 2 4
7 6 5

1 _ 3
8 2 4
7 6 5

1 2 3
8 _ 4
7 6 5



# Water Pouring Problems

![](http://puzzles.nigelcoldwell.co.uk/images/water22.png)

In a [water pouring problem](https://en.wikipedia.org/wiki/Water_pouring_puzzle) you are given a collection of jugs, each of which has a size (capacity) in, say, litres, and a current level of water (in litres). The goal is to measure out a certain level of water; it can appear in any of the jugs. For example, in the movie *Die Hard 3*, the heroes were faced with the task of making exactly 4 gallons from jugs of size 5 gallons and 3 gallons.) A state is represented by a tuple of current water levels, and the available actions are:
- `(Fill, i)`: fill the `i`th jug all the way to the top (from a tap with unlimited water).
- `(Dump, i)`: dump all the water out of the `i`th jug.
- `(Pour, i, j)`: pour water from the `i`th jug into the `j`th jug until either the jug `i` is empty, or jug `j` is full, whichever comes first.

In [25]:
class PourProblem(Problem):
    """Problem about pouring water between jugs to achieve some water level.
    Each state is a tuples of water levels. In the initialization, also provide a tuple of 
    jug sizes, e.g. PourProblem(initial=(0, 0), goal=4, sizes=(5, 3)), 
    which means two jugs of sizes 5 and 3, initially both empty, with the goal
    of getting a level of 4 in either jug."""
    
    def actions(self, state):
        """The actions executable in this state."""
        jugs = range(len(state))
        return ([('Fill', i)    for i in jugs if state[i] < self.sizes[i]] +
                [('Dump', i)    for i in jugs if state[i]] +
                [('Pour', i, j) for i in jugs if state[i] for j in jugs if i != j])

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        result = list(state)
        act, i, *_ = action
        if act == 'Fill':   # Fill i to capacity
            result[i] = self.sizes[i]
        elif act == 'Dump': # Empty i
            result[i] = 0
        elif act == 'Pour': # Pour from i into j
            j = action[2]
            amount = min(state[i], self.sizes[j] - state[j])
            result[i] -= amount
            result[j] += amount
        return tuple(result)

    def is_goal(self, state):
        """True if the goal level is in any one of the jugs."""
        return self.goal in state

In a `GreenPourProblem`, the states and actions are the same, but instead of all actions costing 1, in these problems the cost of an action is the amount of water that flows from the tap. (There is an issue that non-*Fill* actions have 0 cost, which in general can lead to indefinitely long solutions, but in this problem there is a finite number of states, so we're ok.)

In [26]:
class GreenPourProblem(PourProblem): 
    """A PourProblem in which the cost is the amount of water used."""
    def action_cost(self, s, action, s1):
        "The cost is the amount of water used."
        act, i, *_ = action
        return self.sizes[i] - s[i] if act == 'Fill' else 0

In [27]:
# Some specific PourProblems

p1 = PourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
p2 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p3 = PourProblem((0, 0),     8, sizes=(7,9))
p4 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p5 = PourProblem((0, 0),     4, sizes=(3, 5))

g1 = GreenPourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
g2 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g3 = GreenPourProblem((0, 0),     8, sizes=(7,9))
g4 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g5 = GreenPourProblem((0, 0),     4, sizes=(3, 5))

In [28]:
# Solve the PourProblem of getting 13 in some jug, and show the actions and states
soln = breadth_first_search(p1)
path_actions(soln), path_states(soln)

([('Fill', 1), ('Pour', 1, 0), ('Dump', 0), ('Pour', 1, 0)],
 [(1, 1, 1), (1, 16, 1), (2, 15, 1), (0, 15, 1), (2, 13, 1)])

# Pancake Sorting Problems

Given a stack of pancakes of various sizes, can you sort them into a stack of decreasing sizes, largest on bottom to smallest on top? You have a spatula with which you can flip the top `i` pancakes. This is shown below for `i = 3`; on the top the spatula grabs the first three pancakes; on the bottom we see them flipped:


![](https://upload.wikimedia.org/wikipedia/commons/0/0f/Pancake_sort_operation.png)

How many flips will it take to get the whole stack sorted? This is an interesting [problem](https://en.wikipedia.org/wiki/Pancake_sorting) that Bill Gates has [written about](https://people.eecs.berkeley.edu/~christos/papers/Bounds%20For%20Sorting%20By%20Prefix%20Reversal.pdf). A reasonable heuristic for this problem is the *gap heuristic*: if we look at neighboring pancakes, if, say, the 2nd smallest is next to the 3rd smallest, that's good; they should stay next to each other. But if the 2nd smallest is next to the 4th smallest, that's bad: we will require at least one move to separate them and insert the 3rd smallest between them. The gap heuristic counts the number of neighbors that have a gap like this. In our specification of the problem, pancakes are ranked by size: the smallest is `1`, the 2nd smallest `2`, and so on, and the representation of a state is a tuple of these rankings, from the top to the bottom pancake. Thus the goal state is always `(1, 2, ..., `*n*`)` and the initial (top) state in the diagram above is `(2, 1, 4, 6, 3, 5)`.


In [29]:
class PancakeProblem(Problem):
    """A PancakeProblem the goal is always `tuple(range(1, n+1))`, where the
    initial state is a permutation of `range(1, n+1)`. An act is the index `i` 
    of the top `i` pancakes that will be flipped."""
    
    def __init__(self, initial): 
        self.initial, self.goal = tuple(initial), tuple(sorted(initial))
    
    def actions(self, state): return range(2, len(state) + 1)

    def result(self, state, i): return state[:i][::-1] + state[i:]
    
    def h(self, node):
        "The gap heuristic."
        s = node.state
        return sum(abs(s[i] - s[i - 1]) > 1 for i in range(1, len(s)))

In [30]:
c0 = PancakeProblem((2, 1, 4, 6, 3, 5))
c1 = PancakeProblem((4, 6, 2, 5, 1, 3))
c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4))
c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4))
c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8))

In [31]:
# Solve a pancake problem
path_states(astar_search(c0))

Time Taken:  0.0009996891021728516
Nodes Expanded:  32
States Reached:  108
Nodes in Frontier:  76
Solution Cost:  5
Solution Length:  5
Reexpanded States:  0
Dummy Nodes:  0


[(2, 1, 4, 6, 3, 5),
 (6, 4, 1, 2, 3, 5),
 (5, 3, 2, 1, 4, 6),
 (4, 1, 2, 3, 5, 6),
 (3, 2, 1, 4, 5, 6),
 (1, 2, 3, 4, 5, 6)]

# Jumping Frogs Puzzle

In this puzzle (which also can be played as a two-player game), the initial state is a line of squares, with N pieces of one kind on the left, then one empty square, then N pieces of another kind on the right. The diagram below uses 2 blue toads and 2 red frogs; we will represent this as the string `'LL.RR'`. The goal is to swap the pieces, arriving at `'RR.LL'`. An `'L'` piece moves left-to-right, either sliding one space ahead to an empty space, or two spaces ahead if that space is empty and if there is an `'R'` in between to hop over. The `'R'` pieces move right-to-left analogously. An action will be an `(i, j)` pair meaning to swap the pieces at those indexes. The set of actions for the N = 2 position below is `{(1, 2), (3, 2)}`, meaning either the blue toad in position 1 or the red frog in position 3 can swap places with the blank in position 2.

![](https://upload.wikimedia.org/wikipedia/commons/2/2f/ToadsAndFrogs.png)

In [32]:
class JumpingPuzzle(Problem):
    """Try to exchange L and R by moving one ahead or hopping two ahead."""
    def __init__(self, N=2):
        self.initial = N*'L' + '.' + N*'R'
        self.goal = self.initial[::-1]
        
    def actions(self, state):
        """Find all possible move or hop moves."""
        idxs = range(len(state))
        return ({(i, i + 1) for i in idxs if state[i:i+2] == 'L.'}   # Slide
               |{(i, i + 2) for i in idxs if state[i:i+3] == 'LR.'}  # Hop
               |{(i + 1, i) for i in idxs if state[i:i+2] == '.R'}   # Slide
               |{(i + 2, i) for i in idxs if state[i:i+3] == '.LR'}) # Hop

    def result(self, state, action):
        """An action (i, j) means swap the pieces at positions i and j."""
        i, j = action
        result = list(state)
        result[i], result[j] = state[j], state[i]
        return ''.join(result)
    
    def h(self, node): return hamming_distance(node.state, self.goal)

In [33]:
JumpingPuzzle(N=2).actions('LL.RR')

{(1, 2), (3, 2)}

In [34]:
j3 = JumpingPuzzle(N=3)
j9 = JumpingPuzzle(N=9)
path_states(astar_search(j3))

Time Taken:  0.0
Nodes Expanded:  71
States Reached:  72
Nodes in Frontier:  1
Solution Cost:  15
Solution Length:  15
Reexpanded States:  0
Dummy Nodes:  0


['LLL.RRR',
 'LLLR.RR',
 'LL.RLRR',
 'L.LRLRR',
 'LRL.LRR',
 'LRLRL.R',
 'LRLRLR.',
 'LRLR.RL',
 'LR.RLRL',
 '.RLRLRL',
 'R.LRLRL',
 'RRL.LRL',
 'RRLRL.L',
 'RRLR.LL',
 'RR.RLLL',
 'RRR.LLL']

# Reporting Summary Statistics on Search Algorithms

Now let's gather some metrics on how well each algorithm does.  We'll use `CountCalls` to wrap a `Problem` object in such a way that calls to its methods are delegated to the original problem, but each call increments a counter. Once we've solved the problem, we print out summary statistics.

In [35]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, p.str()[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

Here's a tiny report for uniform-cost search on the jug pouring problems:

In [36]:
report([uniform_cost_search], [p1])

uniform_cost_search:
Time Taken:  0.003000974655151367
Nodes Expanded:  109
States Reached:  188
Nodes in Frontier:  79
Solution Cost:  4
Solution Length:  4
Reexpanded States:  0
Dummy Nodes:  0


AttributeError: 'PourProblem' object has no attribute 'str'

In [ ]:
report((uniform_cost_search, breadth_first_search), 
       (p1, g1, p2, g2, p3, g3, p4, g4, p4, g4, c1, c2, c3)) 

uniform_cost_search:
Time Taken:  0.0029985904693603516
Nodes Expanded:  109
States Reached:  188
Nodes in Frontier:  79
Solution Cost:  4
Solution Length:  4
Reexpanded States:  0
Dummy Nodes:  0


AttributeError: 'PourProblem' object has no attribute 'str'

# Comparing heuristics

First, let's look at the eight puzzle problems, and compare three different heuristics the Manhattan heuristic, the less informative misplaced tiles heuristic, and the uninformed (i.e. *h* = 0) breadth-first search:

In [ ]:
def astar_misplaced_tiles(problem): return astar_search(problem, h=problem.h1)

report([astar_misplaced_tiles], 
       [instance_gr17,instance_gr21_commented_4,instance_gr21,instance_fri26_commented_5])

astar_misplaced_tiles:
Time Taken:  10.731019973754883
Nodes Expanded:  94232
States Reached:  257193
Nodes in Frontier:  232631
Solution Cost:  2085
Solution Length:  17
Reexpanded States:  2
Dummy Nodes:  31000
  828,753 nodes |94232 cost |   2,085 actions | 94248
Time Taken:  0.6309998035430908
Nodes Expanded:  3774
States Reached:  24565
Nodes in Frontier:  24065
Solution Cost:  2369
Solution Length:  17
Reexpanded States:  0
Dummy Nodes:  717
   39,726 nodes | 3774 cost |   2,369 actions | 3790
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\elpla\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\elpla\AppData\Local\Temp\ipykernel_1692\2560939167.py", line 3, in <module>
    report([astar_misplaced_tiles],
  File "C:\Users\elpla\AppData\Local\Temp\ipykernel_1692\851020010.py", line 20, in report
    soln   = searcher(prob)
             ^^^^^^^^^^^^^^
  File "C:\Users\elpla\AppData\Local\Temp\ipykernel_1692\2560939167.py", line 1, in astar_misplaced_tiles
    def astar_misplaced_tiles(problem): return astar_search(problem, h=problem.h1)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\elpla\AppData\Local\Temp\ipykernel_1692\3703837168.py", line 73, in astar_search
    return best_first_search(problem, f=lambda n: g(n) + h(n))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\

We see that all three algorithms get cost-optimal solutions, but the better the heuristic, the fewer nodes explored. 
Compared to the uninformed search, the misplaced tiles heuristic explores about 1/4 the number of nodes, and the Manhattan heuristic needs just 2%.

Next, we can show the value of the gap heuristic for pancake sorting problems:

In [ ]:
report([astar_search, uniform_cost_search], [c1, c2, c3, c4])

astar_search:
Time Taken:  0.006997108459472656
Nodes Expanded:  256
States Reached:  508
Nodes in Frontier:  270
Solution Cost:  7
Solution Length:  7
Reexpanded States:  0
Dummy Nodes:  2


AttributeError: 'PancakeProblem' object has no attribute 'str'

We need to explore 300 times more nodes without the heuristic.

# Comparing graph search and tree search

Keeping the *reached* table in `best_first_search` allows us to do a graph search, where we notice when we reach a state by two different paths, rather than a tree search, where we have duplicated effort. The *reached* table consumes space and also saves time. How much time? In part it depends on how good the heuristics are at focusing the search.  Below we show that on some pancake and eight puzzle problems, the tree search expands roughly twice as many nodes (and thus takes roughly twice as much time):

In [ ]:
report([astar_search, astar_tree_search], [e1, e2, e3, e4, r1, r2, r3, r4])

astar_search:
Time Taken:  0.7419984340667725
Nodes Expanded:  34888
States Reached:  49980
Nodes in Frontier:  15627
Solution Cost:  30
Solution Length:  30
Reexpanded States:  118
Dummy Nodes:  637
   94,071 nodes |   34,888 goal |   30 cost |  34,917 actions | EightPuzzle((5, 6, 7, 2, 8, 4, 0, 3, 1),
Time Taken:  0.6810004711151123
Nodes Expanded:  32697
States Reached:  47530
Nodes in Frontier:  15572
Solution Cost:  30
Solution Length:  30
Reexpanded States:  315
Dummy Nodes:  705
   88,327 nodes |   32,697 goal |   30 cost |  32,726 actions | EightPuzzle((5, 6, 7, 4, 0, 8, 3, 2, 1),
Time Taken:  0.9889986515045166
Nodes Expanded:  47165
States Reached:  64914
Nodes in Frontier:  18396
Solution Cost:  30
Solution Length:  30
Reexpanded States:  105
Dummy Nodes:  869
  126,825 nodes |   47,165 goal |   30 cost |  47,194 actions | EightPuzzle((5, 4, 7, 6, 0, 3, 8, 2, 1),
Time Taken:  0.912999153137207
Nodes Expanded:  44081
States Reached:  60954
Nodes in Frontier:  17551
Solution C

# Comparing different weighted search values

Below we report on problems using these four algorithms:

|Algorithm|*f*|Optimality|
|:---------|---:|:----------:|
|Greedy best-first search | *f = h*|nonoptimal|
|Extra weighted A* search | *f = g + 2 &times; h*|nonoptimal|
|Weighted A* search | *f = g + 1.4 &times; h*|nonoptimal|
|A* search | *f = g + h*|optimal|
|Uniform-cost search | *f = g*|optimal|

We will see that greedy best-first search (which ranks nodes solely by the heuristic) explores the fewest number of nodes, but has the highest path costs. Weighted A* search explores twice as many nodes (on this problem set) but gets 10% better path costs. A* is optimal, but explores more nodes, and uniform-cost is also optimal, but explores an order of magnitude more nodes.

In [ ]:
def extra_weighted_astar_search(problem): return weighted_astar_search(problem, weight=2)
    
report((greedy_bfs, extra_weighted_astar_search, weighted_astar_search, astar_search, uniform_cost_search), 
       (r0, r1, r2, r3, r4, e1, d1, d2, j9, e2, d3, d4, d6, d7, e3, e4))

greedy_bfs:
Time Taken:  0.0
Nodes Expanded:  1
States Reached:  1
Nodes in Frontier:  0
Solution Cost:  0
Solution Length:  0
Reexpanded States:  0
Dummy Nodes:  0
        0 nodes |        1 goal |    0 cost |       0 actions | RouteProblem('A', 'A')
Time Taken:  0.0
Nodes Expanded:  4
States Reached:  8
Nodes in Frontier:  4
Solution Cost:  450
Solution Length:  3
Reexpanded States:  0
Dummy Nodes:  0
        9 nodes |        4 goal |  450 cost |       6 actions | RouteProblem('A', 'B')
Time Taken:  0.0
Nodes Expanded:  12
States Reached:  17
Nodes in Frontier:  5
Solution Cost:  1207
Solution Length:  11
Reexpanded States:  0
Dummy Nodes:  0
       28 nodes |       12 goal | 1207 cost |      22 actions | RouteProblem('N', 'L')
Time Taken:  0.0
Nodes Expanded:  8
States Reached:  14
Nodes in Frontier:  6
Solution Cost:  837
Solution Length:  7
Reexpanded States:  0
Dummy Nodes:  0
       19 nodes |        8 goal |  837 cost |      14 actions | RouteProblem('E', 'T')
Time Taken:  0.0


KeyboardInterrupt: 

We see that greedy search expands the fewest nodes, but has the highest path costs. In contrast, A\* gets optimal path costs, but expands 4 or 5 times more nodes. Weighted A* is a good compromise, using half the compute time as A\*, and achieving path costs within  1% or 2% of optimal. Uniform-cost is optimal, but is an order of magnitude slower than A\*.

# Comparing  many search algorithms

Finally, we compare a host of algorihms (even the slow ones) on some of the easier problems:

In [ ]:
report((astar_search, uniform_cost_search,  breadth_first_search, breadth_first_bfs, 
        iterative_deepening_search, depth_limited_search, greedy_bfs, 
        weighted_astar_search, extra_weighted_astar_search), 
       (p1, g1, p2, g2, p3, g3, p4, g4, r0, r1, r2, r3, r4, e1))

astar_search:
Time Taken:  0.003094196319580078
Nodes Expanded:  109
States Reached:  188
Nodes in Frontier:  79
Solution Cost:  4
Solution Length:  4
Reexpanded States:  0
Dummy Nodes:  0
      948 nodes |      109 goal |    4 cost |     112 actions | PourProblem((1, 1, 1), 13)
Time Taken:  0.004986286163330078
Nodes Expanded:  189
States Reached:  291
Nodes in Frontier:  111
Solution Cost:  10
Solution Length:  15
Reexpanded States:  0
Dummy Nodes:  1
    1,688 nodes |      189 goal |   10 cost |     203 actions | GreenPourProblem((1, 1, 1), 13)
Time Taken:  0.012120962142944336
Nodes Expanded:  389
States Reached:  528
Nodes in Frontier:  139
Solution Cost:  9
Solution Length:  9
Reexpanded States:  0
Dummy Nodes:  0
    3,499 nodes |      389 goal |    9 cost |     397 actions | PourProblem((0, 0, 0), 21)
Time Taken:  0.015126943588256836
Nodes Expanded:  448
States Reached:  712
Nodes in Frontier:  338
Solution Cost:  21
Solution Length:  10
Reexpanded States:  0
Dummy Nodes:  6
 

KeyboardInterrupt: 

This confirms some of the things we already knew: A* and uniform-cost search are optimal, but the others are not. A* explores fewer nodes than uniform-cost. 

# Visualizing Reached States

I would like to draw a picture of the state space, marking the states that have been reached by the search.
Unfortunately, the *reached* variable is inaccessible inside `best_first_search`, so I will define a new version of `best_first_search` that is identical except that it declares *reached* to be `global`. I can then define `plot_grid_problem` to plot the obstacles of a `GridProblem`, along with the initial and goal states, the solution path, and the states reached during a search.

In [ ]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    global reached # <<<<<<<<<<< Only change here
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def plot_grid_problem(grid, solution, reached=(), title='Search', show=True):
    "Use matplotlib to plot the grid, obstacles, solution, and reached."
    reached = list(reached)
    plt.figure(figsize=(16, 10))
    plt.axis('off'); plt.axis('equal')
    plt.scatter(*transpose(grid.obstacles), marker='s', color='darkgrey')
    plt.scatter(*transpose(reached), 1**2, marker='.', c='blue')
    plt.scatter(*transpose(path_states(solution)), marker='s', c='blue')
    plt.scatter(*transpose([grid.initial]), 9**2, marker='D', c='green')
    plt.scatter(*transpose([grid.goal]), 9**2, marker='8', c='red')
    if show: plt.show()
    print('{} {} search: {:.1f} path cost, {:,d} states reached'
          .format(' ' * 10, title, solution.path_cost, len(reached)))
    
def plots(grid, weights=(1.4, 2)): 
    """Plot the results of 4 heuristic search algorithms for this grid."""
    solution = astar_search(grid)
    plot_grid_problem(grid, solution, reached, 'A* search')
    for weight in weights:
        solution = weighted_astar_search(grid, weight=weight)
        plot_grid_problem(grid, solution, reached, '(b) Weighted ({}) A* search'.format(weight))
    solution = greedy_bfs(grid)
    plot_grid_problem(grid, solution, reached, 'Greedy best-first search')
    
def transpose(matrix): return list(zip(*matrix))

In [ ]:
plots(d3)

In [ ]:
plots(d4)

# The cost of weighted A* search

Now I want to try a much simpler grid problem, `d6`, with only a few obstacles. We see that A* finds the optimal path, skirting below the obstacles. Weighterd A* with a weight of 1.4 finds the same optimal path while exploring only 1/3 the number of states. But weighted A* with weight 2 takes the slightly longer path above the obstacles, because that path allowed it to stay closer to the goal in straight-line distance, which it over-weights. And greedy best-first search has a bad showing, not deviating from its path towards the goal until it is almost inside the cup made by the obstacles.

In [ ]:
plots(d6)

In the next problem, `d7`, we see a similar story. the optimal path found by A*, and we see that again weighted A* with weight 1.4 does great and with weight 2 ends up erroneously going below the first two barriers, and then makes another mistake by reversing direction back towards the goal and passing above the third barrier. Again, greedy best-first makes bad decisions all around.

In [ ]:
plots(d7)

# Nondeterministic Actions

To handle problems with nondeterministic problems, we'll replace the `result` method with `results`, which returns a collection of possible result states. We'll represent the solution to a problem not with a `Node`, but with a plan that consist of two types of component: sequences of actions, like `['forward', 'suck']`, and condition actions, like
`{5: ['forward', 'suck'], 7: []}`, which says that if we end up in state 5, then do `['forward', 'suck']`, but if we end up in state 7, then do the empty sequence of actions.

In [ ]:
def and_or_search(problem):
    "Find a plan for a problem that has nondterministic actions."
    return or_search(problem, problem.initial, [])
    
def or_search(problem, state, path):
    "Find a sequence of actions to reach goal from state, without repeating states on path."
    if problem.is_goal(state): return []
    if state in path: return failure # check for loops
    for action in problem.actions(state):
        plan = and_search(problem, problem.results(state, action), [state] + path)
        if plan != failure:
            return [action] + plan
    return failure

def and_search(problem, states, path):
    "Plan for each of the possible states we might end up in."
    if len(states) == 1: 
        return or_search(problem, next(iter(states)), path)
    plan = {}
    for s in states:
        plan[s] = or_search(problem, s, path)
        if plan[s] == failure: return failure
    return [plan]

In [ ]:
class MultiGoalProblem(Problem):
    """A version of `Problem` with a colllection of `goals` instead of one `goal`."""
    
    def __init__(self, initial=None, goals=(), **kwds): 
        self.__dict__.update(initial=initial, goals=goals, **kwds)
        
    def is_goal(self, state): return state in self.goals
    
class ErraticVacuum(MultiGoalProblem):
    """In this 2-location vacuum problem, the suck action in a dirty square will either clean up that square,
    or clean up both squares. A suck action in a clean square will either do nothing, or
    will deposit dirt in that square. Forward and backward actions are deterministic."""
    
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = {'suck':{1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
             'forward': {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
             'backward': {1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}}}

Let's find a plan to get from state 1 to the goal of no dirt (states 7 or 8):

In [ ]:
and_or_search(ErraticVacuum(1, {7, 8}))

This plan says "First suck, and if we end up in state 5, go forward and suck again; if we end up in state 7, do nothing because that is a goal."

Here are the plans to get to a goal state starting from any one of the 8 states:

In [ ]:
{s: and_or_search(ErraticVacuum(s, {7,8})) 
 for s in range(1, 9)}

# Comparing Algorithms on EightPuzzle Problems of Different Lengths

In [ ]:
from functools import lru_cache

def build_table(table, depth, state, problem):
    if depth > 0 and state not in table:
        problem.initial = state
        table[state] = len(astar_search(problem))
        for a in problem.actions(state):
            build_table(table, depth - 1, problem.result(state, a), problem)
    return table

def invert_table(table):
    result = defaultdict(list)
    for key, val in table.items():
        result[val].append(key)
    return result

goal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
table8 = invert_table(build_table({}, 25, goal, EightPuzzle(goal)))

In [ ]:
def report8(table8, M, Ds=range(2, 25, 2), searchers=(breadth_first_search, astar_misplaced_tiles, astar_search)):
    "Make a table of average nodes generated and effective branching factor"
    for d in Ds:
        line = [d]
        N = min(M, len(table8[d]))
        states = random.sample(table8[d], N)
        for searcher in searchers:
            nodes = 0
            for s in states:
                problem = CountCalls(EightPuzzle(s))
                searcher(problem)
                nodes += problem._counts['result']
            nodes = int(round(nodes/N))
            line.append(nodes)
        line.extend([ebf(d, n) for n in line[1:]])
        print('{:2} & {:6} & {:5} & {:5} && {:.2f} & {:.2f} & {:.2f}'
              .format(*line))

        
def ebf(d, N, possible_bs=[b/100 for b in range(100, 300)]):
    "Effective Branching Factor"
    return min(possible_bs, key=lambda b: abs(N - sum(b**i for i in range(1, d+1))))

def edepth_reduction(d, N, b=2.67):
    
    

from statistics import mean 

def random_state():
    x = list(range(9))
    random.shuffle(x)
    return tuple(x)

meanbf = mean(len(e3.actions(random_state())) for _ in range(10000))
meanbf

In [ ]:
{n: len(v) for (n, v) in table30.items()}

In [ ]:
%time table30 = invert_table(build_table({}, 30, goal, EightPuzzle(goal)))

In [ ]:
%time report8(table30, 20, range(26, 31, 2))

In [ ]:
%time report8(table30, 20, range(26, 31, 2))

In [ ]:
from itertools import combinations
from statistics import median, mean

# Detour index for Romania

L = romania.locations
def ratio(a, b): return astar_search(RouteProblem(a, b, map=romania)).path_cost / sld(L[a], L[b])
nums = [ratio(a, b) for a,b in combinations(L, 2) if b in r1.actions(a)]
mean(nums), median(nums) # 1.7, 1.6 # 1.26, 1.2 for adjacent cities

In [ ]:
sld